# O Que São Estatísticas?

### Estatísticas no SQL Server, incluindo o Azure SQL Database, são coleções de informações sobre a distribuição de valores de dados em uma ou mais colunas de uma tabela ou exibição indexada. Essas estatísticas são armazenadas como objetos binários grandes (BLOBs) no banco de dados do usuário.

#### Como as Estatísticas Funcionam?

## 

- **Distribuição de Dados**: Estatísticas contêm informações sobre como os dados são distribuídos dentro de uma coluna. Isso inclui detalhes como a frequência dos valores e a distribuição dos intervalos de valores.
- **Cardinalidade**: A cardinalidade refere-se ao número de linhas que uma consulta é esperada retornar. As estatísticas ajudam o otimizador de consultas a estimar a cardinalidade de uma consulta.

#### Papel das Estatísticas no Planejamento de Execução

- **Estimativas de Cardinalidade**: O otimizador de consultas usa estatísticas para calcular a cardinalidade, ou seja, o número esperado de linhas que uma consulta vai retornar. Essas estimativas são fundamentais para o planejamento de execução.
- **Geração do Plano de Execução**: Com base nas estimativas de cardinalidade, o otimizador de consultas decide o melhor plano de execução para uma consulta. Isso inclui a escolha de operações como busca por índice (index seek) ou varredura de índice (index scan).

# Impacto das Estatísticas no Desempenho

####   

1. **Precisão nas Estimativas**: Estatísticas precisas levam a estimativas de cardinalidade precisas, o que resulta em planos de execução mais eficientes.
2. **Escolha das Operações de Acesso**: O otimizador pode escolher entre uma busca por índice (mais eficiente) ou uma varredura de índice (menos eficiente) com base na cardinalidade estimada.
3. **Uso de Recursos**: Planos de execução eficientes utilizam menos recursos, como CPU, memória e I/O, resultando em melhor desempenho geral do banco de dados.

### Exemplos Práticos de Uso de Estatísticas

#### Consulta de Exemplo

In [ ]:
SELECT * 
FROM Sales.Orders 
WHERE OrderDate = '2024-01-01';


- **Cardinalidade Estimada**: O otimizador de consultas usa estatísticas sobre a coluna `OrderDate` para estimar quantas linhas serão retornadas.
- **Plano de Execução**: Com base nessa estimativa, o otimizador decide se deve usar uma busca por índice ou uma varredura de índice.

#### Gerenciamento de Estatísticas

## 

- **Criação de Estatísticas**:

In [ ]:
CREATE STATISTICS OrderDateStats
ON Sales.Orders (OrderDate);


## **Atualização de Estatísticas**: As estatísticas podem se tornar desatualizadas à medida que os dados mudam. O SQL Server pode atualizar automaticamente as estatísticas, mas em alguns casos, pode ser necessário atualizar manualmente.

In [ ]:
UPDATE STATISTICS Sales.Orders (OrderDateStats);


### Boas Práticas no Gerenciamento de Estatísticas

1. **Monitoramento Regular**: Monitorar a precisão das estatísticas regularmente e atualizá-las conforme necessário.
2. **Manutenção Automática**: Configurar a manutenção automática de estatísticas para garantir que elas sejam atualizadas conforme os dados mudam.
3. **Uso de Histogramas**: Entender e utilizar histogramas, que são parte das estatísticas que fornecem uma visão detalhada da distribuição dos dados.
4. **Recompilação de Consultas**: Em alguns casos, pode ser necessário recompilar consultas para garantir que o otimizador utilize as estatísticas mais recentes.

Query para identificar todas as statisticas

In [ ]:
SELECT 
    st.stats_id,
    st.name AS stats_name,
    sp.last_updated,
    sp.rows,
    sp.rows_sampled,
    col.name AS key_column_name
FROM 
    sys.stats st
    JOIN sys.tables tab ON st.object_id = tab.object_id
    CROSS APPLY sys.dm_db_stats_properties(st.object_id, st.stats_id) AS sp
    JOIN sys.stats_columns sc ON st.object_id = sc.object_id AND st.stats_id = sc.stats_id
    JOIN sys.columns col ON sc.object_id = col.object_id AND sc.column_id = col.column_id
WHERE 
    st.user_created = 0
    AND sp.rows > 1000
ORDER BY 
    st.name, col.column_id;
